In [69]:
import sys
sys.path.append('..')

import math
import numpy as np
import scipy.integrate as integrate
import scipy.special

import metrics

# Definiton

Let $f$ a function: $f: \mathbb{R} \to \mathbb{R}$  
The definite integral of $f$ in the interval $[a, b]$ is denoted:

$$\int_a^b f(x)dx$$

It represents the signed area of the region between the x-axis and the function curve of $f$, in the interval $[a, b]$.  

Let $F$ a function: $F: \mathbb{R} \to \mathbb{R}$ such that:

$$\frac{d}{dx}F(x) = f(x) \forall x \in \mathbb{R}$$

$$\int_a^b f(x)dx = [F(x)]_a^b = F(b) - F(a)$$

$F$ can be referred as the integral of $f$, denoted sometimes as:

$$F(x) = \int f(x)dx$$

In [28]:
'''
Apprimate the area under the curve of a between a and b
Suppose f always positive
'''
def est_auc_pos(f, a, b, height, n):
    
    below = 0
    for _ in range(n):
    
        x = np.random.rand() * (b - a) + a
        y = np.random.rand() * height
        
        if y <= f(x): #point below curve
            below += 1
        
    area = (below / n) * (b - a) * height
    return area
        
        

def fconst(c):
    return lambda x: c

v1 = 4 * 3 - 4 *(-5)
v2 = est_auc_pos(fconst(4), -5, 3, 10, 1000000)
print('ref:', v1)
print('est:', v2)

ref: 32
est: 31.93464


In [32]:
def faff(a, b):
    return lambda x: a*x + b

v1 =(7**2 + 3*7) - ((-1)**2 +3 *(-1))
v2 = est_auc_pos(faff(2, 3), -1, 7, 25, 1000000)
print('ref:', v1)
print('est:', v2)

ref: 72
est: 71.99839999999999


In [34]:
'''
Apprimate the area under the curve of a between a and b
'''
def est_auc(f, a, b, height, n):
    
    total_above = 0
    above = 0
    total_below = 0
    below = 0
    for _ in range(n):
    
        x = np.random.rand() * (b - a) + a
        y = np.random.rand() * 2 * height - height
        
        if y >= 0:
            total_above += 1
            if y <= f(x): #point below curve
                above += 1
        else:
            total_below += 1
            if y >= f(x): #point above curve
                below += 1
       
    area_above = (above / total_above) * (b - a) * height
    area_below = (below / total_below) * (b - a) * height
    area = area_above - area_below
    return area

v1 =(7**2 + 3*7) - ((-1)**2 +3 *(-1))
v2 = est_auc(faff(2, 3), -1, 7, 25, 1000000)
print('ref:', v1)
print('est:', v2)

ref: 72
est: 71.9852187397812


In [73]:
def fquad(a, b, c):
    return lambda x: a*x**2 + b*x + c

f = fquad(1, 4, -2)
def fi(x): return x**3/3 + 2*x**2 - 2*x


v1 = fi(4) - fi(-1)
v2 = est_auc(f, -1, 4, 40, 1000000)
print('ref:', v1)
print('est:', v2)

ref: 41.666666666666664
est: 41.659614916462836


# Numerical Integration

Integration can be computed numerically

In [74]:
v1 = fi(4) - fi(-1)
v2, _ = integrate.quad(f, -1, 4)
print(v1)
print(v2)
print((v1-v2)**2)

41.666666666666664
41.66666666666666
5.048709793414476e-29


## Adaptative Quadrature

The algorithm computes the integral of a function on in interval by making approximations on exponentially smallers intervals until it gets a precise enough approximation, and suming all the computed integrals.  

It's based on the trapezoidal rule, that approximates the area under the curve by approximating the functions as linear parts. The area under the curve can then be calculated by computing the area of the estimated trapezoids.

The trapezoidal rule on n points is defined by:

$$\int_a^b f(x)dx \approx \frac{b-a}{2*n} (f(x_0) + 2 * \sum_{i=1}^{n-1} f(x_i) + f(x_n))$$

with $x_0,\text{...},x_n$ points evenly distributed between $a$ and $b$

For $n = 1$:

$$\int_a^b f(x)dx \approx \frac{b-a}{2} (f(a) + f(b))$$

For $n = 2$:

$$\int_a^b f(x)dx \approx \frac{b-a}{4} (f(a) + 2f((a+b)/2) + f(b))$$

We can compute the error between the 2 approximations, and if it's small enough, return the second, more precise approximation.  
Otherwhise we half the $[a, b]$ interval, recurvisely compute the integral of the 2 sub-intervals, and add the twos to get the final result

In [76]:
def adapt_quad(f, a, b, tol=1e-6):
    m = (a + b) / 2
    
    i1 = (b - a)/2 * (f(a) + f(b))
    i2 = (b - a)/4 * (f(a) + 2 * f(m) + f(b))
    
    if np.abs(i1 - i2) < (b - a) * tol:
        return i2
    else:
        return adapt_quad(f, a, m, tol) + adapt_quad(f, m, b, tol)
    
    
v1 = fi(4) - fi(-1)
v2 = adapt_quad(f, -1, 4)
print(v1)
print(v2)
print((v1-v2)**2)

41.666666666666664
41.6666679084301
1.5419764289726684e-12


In [77]:
def f(x): return np.cos(2*x**2 + 0.3*x -0.2) - 0.4*x**3 + 1.001**x + np.log(0.3 + 3.1*x**2) -1.4


v1, _ = integrate.quad(f, -7, 10)
v2 = adapt_quad(f, -7, 10)
print('ref:', v1)
print('est:', v2)
print((v1-v2)**2)

ref: -705.1022457795194
est: -705.1022459191877
1.9507222622462592e-14


In [79]:
## Gauss errror function

def my_erf(x):
    f = lambda x: math.exp(-x**2)
    i = adapt_quad(f, 0, x)
    return 2 / math.sqrt(math.pi) * i
    

v1 = scipy.special.erf(0.4)
v2 = my_erf(0.4)
print('ref:', v1)
print('est:', v2)
print((v1-v2)**2)

ref: 0.42839235504666845
est: 0.42839228030072246
5.5869564416738084e-15
